In [ ]:
# This is predecessor code for reading in FITS images, finding centers of PSFs, 
# and calculating the image scales, pairwise

# Created 2021 June 25 by E.S.

In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import photutils
from photutils import DAOStarFinder
import numpy as np
import pandas as pd
import glob
import os
from itertools import combinations

In [2]:
def find_psf_centers(imagePass,fwhmPass,thresholdPass):
    '''
    Employ DAOPhot to find PSF coordinates
    
    RETURNS:
    ptsEmpiricalPass: empirically-found points
    '''
    daofind = DAOStarFinder(fwhm=fwhmPass, threshold=thresholdPass, exclude_border=True)
    sources = daofind(imagePass)

    # put the x,y coordinates into an array
    ptsEmpiricalPass = np.transpose(np.concatenate(([sources['xcentroid']], [sources['ycentroid']]),axis=0))

    return ptsEmpiricalPass

In [3]:
stem = "./data/fpr_0390/"
file_list = np.sort(glob.glob(stem + "*"))

In [4]:
print(file_list)

['./data/fpr_0390/epoch_1' './data/fpr_0390/epoch_2'
 './data/fpr_0390/epoch_3']


In [5]:
# initialize dictionary
dict_data = {"file_name": [], "x_pix": [], 
                  "y_pix": []}
'''
dict_data = {"file_name": [], "x_mm": [], 
                  "y_mm": [], "x_pix": [], 
                  "y_pix": []}
'''
for fits_num in range(0,len(file_list)):
    
    # read in reduced FITS file
    hdul = fits.open(file_list[fits_num])
    image_array = hdul[1].data
    
    # choose slice 10 (kind of arbitrary, but is bright)
    array_to_examine = image_array[10,:,:]
    
    # find centers of PSFs
    pt_centers = find_psf_centers(array_to_examine,fwhmPass=3,thresholdPass=1000)
    pt_centers = np.squeeze(pt_centers)
    
    print("Found PSF center:")
    print(pt_centers)
    
    # append
    dict_data["file_name"].append(os.path.basename(file_list[fits_num]))
    dict_data["x_pix"].append(pt_centers[0])
    dict_data["y_pix"].append(pt_centers[1])
    #dict_data["x_mm"].append(np.nan)
    #dict_data["y_mm"].append(np.nan)
    
# convert to DataFrame
df_data = pd.DataFrame.from_dict(dict_data)

Found PSF center:
[143.30151817 146.08948466]
Found PSF center:
[147.14154179 137.26968743]
Found PSF center:
[155.58580212 140.98551874]
Found PSF center:
[151.83255331 149.53582206]
Found PSF center:
[143.72506864 146.03749073]
Found PSF center:
[147.44999823 137.51074196]
Found PSF center:
[156.01909407 141.22858914]
Found PSF center:
[152.22122472 149.79906715]
Found PSF center:
[143.72897219 145.96909438]
Found PSF center:
[147.47342283 137.42411742]
Found PSF center:
[156.03379074 141.14123762]
Found PSF center:
[152.24742032 149.72264023]


In [6]:
df_data

,file_name,x_pix,y_pix
0,S20210624E0014_spdc.fits,143.301518,146.089485
1,S20210624E0015_spdc.fits,147.141542,137.269687
2,S20210624E0016_spdc.fits,155.585802,140.985519
3,S20210624E0017_spdc.fits,151.832553,149.535822
4,S20210625E0002_spdc.fits,143.725069,146.037491
5,S20210625E0003_spdc.fits,147.449998,137.510742
6,S20210625E0004_spdc.fits,156.019094,141.228589
7,S20210625E0005_spdc.fits,152.221225,149.799067
8,S20210625E0009_spdc.fits,143.728972,145.969094
9,S20210625E0010_spdc.fits,147.473423,137.424117


In [7]:
# fake data

dict_data = {"file_name": ["yada1","yada2","yada3","daya4"],
             "x_mm": [1.6,8.5,4.3,5.5], 
             "y_mm": [5.3,2.7,7.8,9.7], 
             "x_pix": [43,67,76,45], 
             "y_pix": [76,45,76,56],}

df_data = pd.DataFrame.from_dict(dict_data)

In [8]:
df_data

,file_name,x_mm,y_mm,x_pix,y_pix
0,yada1,1.6,5.3,43,76
1,yada2,8.5,2.7,67,45
2,yada3,4.3,7.8,76,76
3,daya4,5.5,9.7,45,56


In [9]:
# find all baselines and put into new dataframe

# get all combinations of endpoints, in sets of 2
perm = combinations(np.arange(len(df_data)), 2)

dict_baselines = {}

dict_baselines = {"pt_1_name": [], "pt_2_name": [], 
                  "pt_1_x_pix": [], "pt_1_y_pix": [], 
                  "pt_2_x_pix": [], "pt_2_y_pix": [],
                  "pt_1_x_mm": [], "pt_1_y_mm": [], 
                  "pt_2_x_mm": [], "pt_2_y_mm": []}

# append names to dict
for i in list(perm):
    
    dict_baselines["pt_1_name"].append(df_data["file_name"].iloc[i[0]])
    dict_baselines["pt_2_name"].append(df_data["file_name"].iloc[i[1]])
    dict_baselines["pt_1_x_pix"].append(df_data["x_pix"].iloc[i[0]])
    dict_baselines["pt_2_x_pix"].append(df_data["x_pix"].iloc[i[1]])
    dict_baselines["pt_1_y_pix"].append(df_data["y_pix"].iloc[i[0]])
    dict_baselines["pt_2_y_pix"].append(df_data["y_pix"].iloc[i[1]])
    dict_baselines["pt_1_x_mm"].append(df_data["x_mm"].iloc[i[0]])
    dict_baselines["pt_2_x_mm"].append(df_data["x_mm"].iloc[i[1]])
    dict_baselines["pt_1_y_mm"].append(df_data["y_mm"].iloc[i[0]])
    dict_baselines["pt_2_y_mm"].append(df_data["y_mm"].iloc[i[1]])

df_baselines = pd.DataFrame.from_dict(dict_baselines)

In [10]:
# find distances in pixels, and in ASU movements in mm

dict_baselines["piece_x_pix"] = np.power(np.subtract(dict_baselines["pt_1_x_pix"],dict_baselines["pt_2_x_pix"]),2.)
dict_baselines["piece_y_pix"] = np.power(np.subtract(dict_baselines["pt_1_y_pix"],dict_baselines["pt_2_y_pix"]),2.)
dict_baselines["d_pix"] = np.sqrt(np.add(dict_baselines["piece_x_pix"],dict_baselines["piece_y_pix"]))

dict_baselines["piece_x_mm"] = np.power(np.subtract(dict_baselines["pt_1_x_mm"],dict_baselines["pt_2_x_mm"]),2.)
dict_baselines["piece_y_mm"] = np.power(np.subtract(dict_baselines["pt_1_y_mm"],dict_baselines["pt_2_y_mm"]),2.)
dict_baselines["d_mm"] = np.sqrt(np.add(dict_baselines["piece_x_mm"],dict_baselines["piece_y_mm"]))

In [12]:
# convert mm distances to asec 
# GPI input window is 1.610 arcsec/mm

dict_baselines["d_asec"] = np.multiply(1.610,dict_baselines["d_mm"]) # Gemini input image scale

In [13]:
# add image scales

dict_baselines["image_scale"] = np.divide(dict_baselines["d_asec"],dict_baselines["d_pix"]) # units asec/pix

In [14]:
# average the plate scales

ps_avg = np.mean(dict_baselines["image_scale"])
ps_std = np.std(dict_baselines["image_scale"])

print("PS avg (asec/pix): " + str(ps_avg))
print("PS std (asec/pix): " + str(ps_std))

PS avg (asec/pix): 0.31323051698800125
PS std (asec/pix): 0.1436125686637536
